In [70]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from collections import defaultdict
from textwrap import wrap
from transformers import  AdamW, AutoModelForSequenceClassification, AutoTokenizer
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
# Random seed for reproducibilty
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
# Set GPU
device = "cpu" #torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
class_names = ['negative', 'positive']

MAX_LEN = 512
# Number of iterations 
EPOCHS = 3
# Create train, test and val data loaders
BATCH_SIZE = 8
pdf_src = '../IMDB-Dataset-GoogleTranslate-proccessed-nefnir.csv'
MODEL_NAME = "Mideind/IceBERT"
df = pd.read_csv(pdf_src)
df.drop(['Unnamed: 0'], axis=1, inplace=True)

def convert(sentiment):
    return 1 if sentiment == 'positive' else 0

df['sentiment'] = df.sentiment.apply(convert)

df_train, df_test = train_test_split(df, test_size=0.3, random_state=RANDOM_SEED)
df_val, df_test = train_test_split(df_test, test_size=0.5, random_state=RANDOM_SEED)

model = AutoModelForSequenceClassification.from_pretrained("./sentiment_model/", num_labels=2)
#model.to(device)
tokenizer = AutoTokenizer.from_pretrained("./sentiment_model/", num_labels=2)
model.eval()


model2 = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)
#model2.to(device)
tokenizer2 = AutoTokenizer.from_pretrained(MODEL_NAME, num_labels=2)
model2.eval()


cpu


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at Mideind/IceBERT and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50000, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [73]:
model2.load_state_dict(torch.load("best_model_state.bin"), map_location=torch.device('cpu'))

OutOfMemoryError: CUDA out of memory. Tried to allocate 148.00 MiB. GPU 0 has a total capacty of 3.94 GiB of which 39.88 MiB is free. Process 3580 has 576.00 MiB memory in use. Including non-PyTorch memory, this process has 2.59 GiB memory in use. Of the allocated memory 2.39 GiB is allocated by PyTorch, and 143.75 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [67]:
def predict(txt, model, tokenizer):
    inputs = tokenizer.encode_plus(
            txt,
            max_length=MAX_LEN,
            add_special_tokens=True,
            return_token_type_ids=False,
            pad_to_max_length=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)
    output = model(input_ids, attention_mask)
    prediction = torch.max(output.logits, dim=1)
    return 1 if class_names[prediction.indices] == 'positive' else 0


correct = 0
total = 7500
print(df_val.shape)
print(df_test.shape)
for data in df_test.itertuples():
    prediction = predict(data.review, model, tokenizer)
    if prediction == data.sentiment:
        correct += 1

print(f'Accuracy: {100 * correct / total}')


(7500, 2)
(7500, 2)


/home/olafurj/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Accuracy: 88.02666666666667
